In [55]:
import numpy as np

d_size = 128
def data_generate(d_size = 128):
    x_train = np.array([[np.random.uniform(-1,1),np.random.uniform(-1,1)]])
    if x_train[0][0]*x_train[0][0] < x_train[0][1]:
        label = np.array([[0]])
    else:
        label = np.array([[1]])
    labels = label
    for i in range(d_size-1):
        data = np.array([[np.random.uniform(-1,1),np.random.uniform(-1,1)]])
        x_train = np.concatenate((x_train,data))
        if data[0][0]*data[0][0] < data[0][1]:
            label = np.array([[0]])
        else:
            label = np.array([[1]])
        labels = np.concatenate((labels,label))
    return x_train,labels

x_train, labels = data_generate()
print(x_train.shape)


(128, 2)


In [56]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

print(sigmoid(0))
    

0.5


In [57]:
#two layer Network
class TwoLayerNet():
    def __init__(self):
        self.params = {}
        self.params['w1'] = np.random.randn(2, 2)# input - 2 , hidden 2
        self.params['b1'] = np.zeros((2,1))
        self.params['w2'] = np.random.randn(1,2) # hidden2 , output - 1
        self.params['b2'] = np.zeros((1,1))

    def propagation(self, x):
        z1 = np.dot(self.params["w1"],x) + self.params["b1"]
        a1 = sigmoid(z1)
        z2 = np.dot(self.params["w2"],a1) + self.params["b2"]
        a2 = sigmoid(z2)
        #print(a2)
        return a1,a2

    def cost(self,a2,y):
        m = y.shape[1]

        cross = np.multiply(np.log(a2), y) + np.multiply(np.log(1-a2) , (1-y))
        cost = - np.sum(cross) / m
        cost = np.squeeze(cost)
        return cost

    def back_propagation(self, a1, a2, x, y):

        dz2 = a2 - y
        dw2 = np.dot(dz2, a1.T) / d_size
        db2 = np.sum(dz2, axis=1, keepdims=True) / d_size
        d_gz = 1 - np.power(a1, 2)
        dz1 = np.dot(self.params["w2"].T, dz2) * d_gz
        dw1 = np.dot(dz1, x.T) / d_size
        db1 = np.sum(dz1, axis=1, keepdims=True) / d_size

        grads = {"w1": dw1,
                 "w2": dw2,
                 "b2": db2,
                 "b1": db1}
        return grads

    def update_parameters(self, grads, lr=0.1):
        for k in self.params.keys():
            self.params[k] -= lr * grads[k]

    def training(self, x, y, num_iter,lr=0.1):
        for i in range(num_iter):
            # propagation
            a1, a2 = self.propagation(x)

            # back_propagation
            grads = self.back_propagation(a1, a2, x, y)

            self.update_parameters(grads, lr=lr)
            '''if (i % 10 == 0):
                print(i,self.cost(a2,y) ,self.predict(x))'''

    def predict(self, x,y):
        a1, a2 = self.propagation(x)
        predictions = abs(a2-y) < 0.5
        #print(predictions)

        return np.mean(predictions)

In [58]:
model = TwoLayerNet()
x,y = data_generate()
model.predict(x.T,y.T)


0.625

In [59]:
for i in range(10):
    x_train, y_train = data_generate()
    model.training(x_train.T,y_train.T,num_iter=5000,lr =0.5)
    x_valid, y_valid = data_generate()
    acc = model.predict(x_valid.T,y_valid.T)
    print(i,acc)

0 0.796875


c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


1 0.828125
2 0.875
3 0.859375
4 0.8125
5 0.8359375
6 0.8671875
7 0.8203125
8 0.828125
9 0.8125


In [60]:
import tensorflow as tf

x_train,y_train = data_generate()

X = tf.placeholder(tf.float32, shape=[None,2])
Y = tf.placeholder(tf.float32, shape=[None,1])

w1 = tf.Variable(tf.random_normal([2,2]))
b1 = tf.Variable(tf.random_normal([2,1]))
w2 = tf.Variable(tf.random_normal([2,1]))
b2 = tf.Variable(tf.random_normal([1]))

hypothesis = tf.sigmoid(tf.matmul(tf.sigmoid(tf.matmul(X,w1)),w2) + b2)

cost = - tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) * tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))


In [61]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        x_train,y_train = data_generate()
        for step in range(5000):
            cost_val , _ = sess.run([cost,train],feed_dict ={X : x_train, Y : y_train})
            #if step % 1000 == 0:
             #   print(step,cost_val)

        h,c,a = sess.run([hypothesis, predicted, accuracy], feed_dict = {X: x_train,Y : y_train})
        print(i,"result : ",a)

0 result :  0.7421875
1 result :  0.953125
2 result :  0.875
3 result :  0.859375
4 result :  0.90625
5 result :  0.921875
6 result :  0.8828125
7 result :  0.9375
8 result :  0.9453125
9 result :  0.9609375
